In [1]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from numba import njit
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [2]:
symbol = 'aapl'
data = vbt.AlpacaData.download(symbol,
                               start='2022-1-01',
                               end='2023-3-16',
                               limit=2000000,
                               timeframe='5m').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


FigureWidget({
    'data': [{'close': array([176.61, 176.45, 176.54, ..., 152.58, 152.49, 152.49]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([176.8 , 176.66, 176.54, ..., 152.65, 152.59, 152.54]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([176.61, 176.22, 176.36, ..., 152.54, 152.49, 152.46]),
              'name': 'Candlestick',
              'open': array([176.72, 176.66, 176.37, ..., 152.65, 152.59, 152.48]),
              'type': 'candlestick',
              'uid': '1466f854-d605-4ffc-9b23-1504380dec2d',
              'x': array([datetime.datetime(2022, 1, 3, 9, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 1, 3, 9, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 1, 3, 9, 10, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 15, 23, 45, tzinfo=datetime.timezone.utc),
             

In [3]:
bullish_engulfing = talib.CDLENGULFING(data['Open'], data['High'], data['Low'], data['Close'])
RSI = vbt.IndicatorFactory.from_talib('RSI')
SMA = vbt.IndicatorFactory.from_talib('SMA')

@njit
def buy_signal(close, sma_small, sma_medium, sma_large, bullish_engulfing, rsi, rsi_buy):
    buy = np.where((close>sma_small) & (sma_small>sma_medium) & (sma_medium>sma_large) & (bullish_engulfing==100) & (rsi > rsi_buy), 1, 0)
    return buy

@njit
def stop_signal(low, high, stop_multiplier, close):
    stop_loss = -1*(((low - (high-low)*stop_multiplier)-close)/close)
    return stop_loss

@njit
def profit_signal(high, low, profit_multiplier, close):
    profit = ((high + ((high-low)*profit_multiplier)-close)/close)
    return profit

def three_ma(close, high, low, bullish_engulfing, rsi_window = 14, sma_window_large = 200, sma_window_medium = 50, sma_window_small = 21, rsi_buy = 50, profit_multiplier = 4, stop_multiplier = 2):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    sma_large = SMA.run(close, sma_window_large).real.to_numpy()
    sma_medium = SMA.run(close, sma_window_medium).real.to_numpy()
    sma_small = SMA.run(close, sma_window_small).real.to_numpy()

    entries = buy_signal(close, sma_small, sma_medium, sma_large, bullish_engulfing, rsi, rsi_buy) == 1


    return entries, profit_signal(high, low, profit_multiplier, close), stop_signal(low, high, stop_multiplier, close)


three_ma = vbt.IndicatorFactory(
    class_name = "three_ma",
    short_name = "three_ma",
    input_names = ['close', 'high', 'low', 'bullish_engulfing'],
    output_names = ['buy', 'profit', 'stop_loss'],
    param_names=['rsi_window', 'sma_window_large', 'sma_window_medium', 'sma_window_small', 'rsi_buy', 'profit_multiplier', 'stop_multiplier']
).from_apply_func(three_ma, rsi_window = 14, sma_window_large = 200, sma_window_medium = 50, sma_window_small = 21, rsi_buy = 50, profit_multiplier = 4, stop_multiplier = 2)



sma_windows = np.arange(15,25,step=3,dtype=int)

master_returns = []

for window in sma_windows:
    order = three_ma.run(
        data['Close'],
        data['High'],
        data['Low'],
        bullish_engulfing,
        rsi_window = 14,
        sma_window_large = 200,
        sma_window_medium = 50,
        sma_window_small = window,
        rsi_buy = np.arange(35,65,step=5,dtype=int),
        profit_multiplier =  np.arange(2,5,step=.1,dtype=float),
        stop_multiplier = np.arange(.5,2.5,step=.1,dtype=float),
        param_product = True,
        to_2d = False
    )

    pf = vbt.Portfolio.from_signals(data['Close'], order.buy, sl_stop = order.stop_loss, tp_stop = order.profit, init_cash=10000, freq='5m')
    master_returns.append(pf.total_return())

print(master_returns)
#orders = pf.orders
#returns = pf.total_return()
#print(returns)
#print(returns.max())


[three_ma_rsi_window  three_ma_sma_window_large  three_ma_sma_window_medium  three_ma_sma_window_small  three_ma_rsi_buy  three_ma_profit_multiplier  three_ma_stop_multiplier
14                   200                        50                          15                         40                2.0                         0.5                         0.233903
                                                                                                                                                     0.6                         0.270583
                                                                                                                                                     0.7                         0.217390
                                                                                                                                                     0.8                         0.209439
                                                                                 